# Backend Usage

## Installation

We wanted to make sure Molar is easy to use and to install. That's why our recommended method for installing it is to use docker-compose (see Local installation). That being said, it is also possible to install Molar without using docker (see Remote installation). If you have any use cases that doesn't fit any of these options, feel free to reach us.

### Local installation

To install the backend, you need [Docker](https://docs.docker.com/engine/install/) installed and Molar with some extra dependancies:

```bash
 $ pip install molar[backend]
```

Then, if you wish to install Molar locally, all you need to do is

```bash
 $ molarcli install local
```

This command will create a folder structure containing the data from postgresql, a `docker-compose.yml` file, a `.env` file containing the configuration and finally an alembic folder. Then, it spins up two docker images: `postgresql` and `molar`. Once this process is done, you should be able to access the backend at the address defined.


### Keeping Molar up to date

Since we are using Docker and docker-compose, it is fairly easy to update the backend. The following command must run from the folder where Molar was installed.

```bash
 $ docker-compose pull backend
 $ docker-compose up -d backend
```

### Uninstalling Molar

To uninstall Molar and remove all the data.

```bash
 $ docker-compose down
 $ cd .. && rm -fr molar_data_dir
```

### Remote installation

It is also possible to install Molar on a remote database and not using Docker at all. Note that you still need admin access to the database.

The first step is to install Molar on the remote database:

```bash
 $ molarcli install remote
```

Then the backend can be started using:

```bash
 $ SERVER_HOST=http://localhost \
   POSTGRES_SERVER=postgres_hostname \
   POSTGRES_USER=postgres \
   POSTGRES_PASSSWORD=postgres_password \
   uvicorn molar.backend.main:app
```

There are a couple more environment variables that can be set. Those are listed in the next section.

### Backend settings

This is the list of environment variables that can be set to configure Molar.

| Environment Variable | Default value |  Description |
| --- | --- | --- |
| SERVER_HOST |  | Hostname where the backend is running |
| POSTGRES_SERVER |  | Hostname of the PostgreSQL server |
| POSTGRES_USER   |  | PostgreSQL user |
| POSTGRES_PASSWORD |  | PostgreSQL password |
| EMAILS_ENABLED | false | Whether the backend can send email (registeration etc) |
| SMTP_TLS | false | Whether to use TLS to connect to the smtp server | 
| SMTP_HOST |  | Hostname of the SMTP server to use |
| SMTP_PORT | 25 | Port of the SMTP server | 
| SMTP_USER |  | username to use  smtp server
| SMTP_PASSWORD |  | password
| EMAILS_FROM_EMAIL |  | Email used to send email |
| EMAILS_FROM_NAME |  | Name used to send email |


## Managing different database structure

In order to support different database structure, Molar relies on [Alembic](https://alembic.sqlalchemy.org). Alembic is a migration tool for databases written in Python (think of it as a `git` for database structure). We integrated alembic command line interface in `molarcli` and you can reach most alembic commands through it.

```bash
 $ molarcli alembic --help
```

Molar is shipped with a few database revisions that are branched together.



```
             ---> molar-main
            /
           /
 user-table 
           \                     ---> compchem   
            \                   /
             ----> eventsourcing
                                \
                                 ---> branchpoint for new structure
```

 - `user-table` implements the table to hold the user data (username, password). 
 - `eventsourcing` implements the event sourcing code, it is the base revision for all database managed by Molar. Its revision id is `f31c7d486f1f`.
 - `compchem` is a structure that is shipped with Molar for computational chemistry.
 - `molar-main` is used to store all the database creation request, these requests are not managed through an event store, but rather directly through active records. This revision is not meant to be used outside in any other way.

### Creating a new structure

To create a new database structure, we need to create an alembic revision that will use the eventsourcing (`f31c7d486f1f`) as a starting point.

```bash
 $ molarcli alembic revision -m "revision_name" --branch-label "revision_name" --head f31 --splice
```

This will generate a new file in `molar_data_dir/migrations` that will contain all the code for the new version. The migration files contains two python function: `upgrade` and `downgrade`. The first one is used to defined all the new changes to make, and the second to undo them if needed. For more information, see the [Alembic documentation](https://alembic.sqlalchemy.org/en/latest/tutorial.html#create-a-migration-script).

Once the migration file is created, users will be able to use this version to make new database requests.

In order to work with the event sourcing, the tables needs to have 3 columns at least.
 - a `tablename_id` column of type UUID (sqlalchemy.dialects.postgresql.UUID) which needs to be the primary key as well
 - two columns `created_on` and `updated_on` of type DateTime (sqlalchemy.DateTime)
 
 Here is an example coming from the molecule table of the compchem structure:

In [ ]:
op.create_table(
    "molecule",
    sa.Column("molecule_id", pgsql.UUID, primary_key=True, nullable=False),
    sa.Column("created_on", sa.DateTime, nullable=False),
    sa.Column("updated_on", sa.DateTime, nullable=False),
    sa.Column("metadata", pgsql.JSONB),
    sa.Column("smiles", sa.Text, nullable=False, unique=True),
    schema="public",
)